In [2]:
import pandas as pd
import glob

import matplotlib.pyplot as plt


import seaborn as sns

import datetime
import numpy as np

from helper import read_files

files = glob.glob("data/*.csv")

columns = ['sku', 'brand', 'cat', 'bill', 'store', 'date', 'type', 'mrp', 'price', 'qty']

df_dict = read_files(files, columns)

df = pd.concat(df for df in df_dict.values())

df['store'] = df['store'].apply(lambda x: x[-1])

In [3]:
# Only subset Store 5 data

df = df[df['store'] == '5']

df = df.sort_values(by='date').reset_index()

In [4]:
df.loc[df['brand'].isnull(), 'brand'] = 'UnkBRAND'

df.loc[df['cat'].isnull(), 'cat'] = 'UnkCAT'

In [5]:
df['date'] = pd.to_datetime(df['date'])

df['day'] = pd.DatetimeIndex(df['date']).day

df['month'] = pd.DatetimeIndex(df['date']).month_name()

df['month_year'] = df.date.dt.to_period('M')

df['dayofweek'] = pd.DatetimeIndex(df['date']).day_name()

weekend_days = ['Saturday', 'Sunday']

df['weekend'] = np.where(df['dayofweek'].isin(weekend_days), 1, 0)

df['weekno'] = df.date.dt.week

df = df[df['qty'] != 0]

In [6]:
df['mrp'] = np.where(df.price > df.mrp, df.price, df.mrp)

df =  df[df['mrp'] != 0]

df = df[df['type'] == 'Sale']

In [7]:
df['total'] = df['price'] * df['qty']

df['discount'] = df['mrp'] - df['price']

df['perc_discount'] = ((df['mrp'] - df['price']) / df['mrp']) * 100

df['perc_discount'] = np.where(df['perc_discount'] == -np.inf, 0, df['perc_discount'])

In [8]:
from helper import week_number_from_start

df['weeks_from_start'] = 0

df = week_number_from_start(df)